In [94]:
import numpy as np
import chainer
import cupy
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
import time
import random
# 
# from chainer.links import VGG16Layers
# from chainer.links import ResNet50Layers
# from chainer.links import ResNet152Layers
# from chainer.links import GoogLeNet
# 
# chainer.cuda.get_device_from_id(0).use()

In [49]:
import chainer
import cupy

chainer.__version__, chainer.cuda.available, time.sleep(0.01)

('3.3.0', True, None)

In [16]:
class TestNet(chainer.Chain):
    def __init__(self, class_num):
        super(TestNet, self).__init__()
        with self.init_scope():
            self.fc1 = L.Linear(None, 256)
            self.fc2 = L.Linear(None, 256)
            self.fc3 = L.Linear(None, class_num)
            
    def __call__(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

In [53]:
class LeNet5(chainer.Chain):
    def __init__(self, class_cnt):
        super(LeNet5, self).__init__()
        with self.init_scope():
            self.c1 = L.Convolution2D(None, 6, 5, 1, 2) # >28 > 14
            self.c2 = L.Convolution2D(None, 16, 5, 1) # 10 > 5
            #self.c3 = L.Convolution2D(None, 120, 4, 1)  # 1
            self.fc1 = L.Linear(None, 120)
            self.fc2 = L.Linear(None, 84)
            self.fc3 = L.Linear(None, class_cnt)
            
    def __call__(self, x):
        x = F.sigmoid(self.c1(x))
        x = F.max_pooling_2d(x, 2, 2)
        x = F.sigmoid(self.c2(x))
        x = F.max_pooling_2d(x, 2, 2)
        #x = F.sigmoid(self.c3(x))
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        return self.fc3(x)
        #if chainer.config.train:
        #    return self.fc2(x)
        #return F.softmax(self.fc2(x))

In [95]:
class LeNetX(chainer.Chain):
    def __init__(self, class_cnt):
        random.seed(0)
        np.random.seed(0)
        cupy.random.seed(0)
        super(LeNetX, self).__init__()

        with self.init_scope():
            self.c1 = L.Convolution2D(None, 6, 5, 1, 2) # >28 > 14
            self.c2 = L.Convolution2D(None, 16, 5, 1) # 10 > 5
            #self.c3 = L.Convolution2D(None, 120, 4, 1)  # 1
            self.fc1 = L.Linear(None, 120)
            self.fc2 = L.Linear(None, 84)
            self.fc3 = L.Linear(None, class_cnt)
            
    def __call__(self, x):
        x = F.relu(self.c1(x))
        x = F.max_pooling_2d(x, 2, 2)
        x = F.relu(self.c2(x))
        x = F.max_pooling_2d(x, 2, 2)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)


In [96]:
class LeNetC(chainer.Chain):
    def __init__(self, class_cnt):
        random.seed(0)
        np.random.seed(0)
        cupy.random.seed(0)
        super(LeNetC, self).__init__()
        with self.init_scope():
            self.c1 = L.Convolution2D(None, 6, 5, 1, 0) # >28 > 14
            self.c2 = L.Convolution2D(None, 16, 5, 1) # 10 > 5
            #self.c3 = L.Convolution2D(None, 120, 4, 1)  # 1
            self.fc1 = L.Linear(None, 120)
            self.fc2 = L.Linear(None, 84)
            self.fc3 = L.Linear(None, class_cnt)
            
    def __call__(self, x):
        x = F.relu(self.c1(x))
        x = F.max_pooling_2d(x, 2, 2)
        x = F.relu(self.c2(x))
        x = F.max_pooling_2d(x, 2, 2)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)


In [111]:
class AlexLike(chainer.Chain):
    insize = 32
    
    def __init__(self):
        random.seed(0)
        np.random.seed(0)
        cupy.random.seed(0)
        
        super(AlexLike, self).__init__()
        with self.init_scope():
            self.c1 = L.Convolution2D(None, 48, 7, 1, 1) # >28 > 14
            self.c2 = L.Convolution2D(None, 128, 5, 1, 1) # >14 > 12
            self.c3 = L.Convolution2D(None, 196, 3, 1, 1) # >12
            self.c4 = L.Convolution2D(None, 196, 3, 1, 1) # >12
            self.c5 = L.Convolution2D(None, 128, 3, 1) # >10 > 5
            self.fc1 = L.Linear(None, 1024) # 3200 > 1024
            self.fc2 = L.Linear(None, 1024)
            self.fc3 = L.Linear(None, 10)
            
    def __call__(self, x):
        x = F.local_response_normalization(F.relu(self.c1(x)))
        x = F.max_pooling_2d(x, 3, 2)
        x = F.local_response_normalization(F.relu(self.c2(x)))
        # x = F.max_pooling_2d(x, 3, 2)
        x = F.relu(self.c3(x))
        x = F.relu(self.c4(x))
        x = F.relu(self.c5(x))
        x = F.max_pooling_2d(x, 3, 2)
        x = F.dropout(F.relu(self.fc1(x)))
        x = F.dropout(F.relu(self.fc2(x)))
        x = F.relu(self.fc3(x))
        return x


## Optimizer
model = TestNet(2)
optimizer = chainer.optimizers.SGD()
optimizer.setup(model)
optimizer.add_hook(chainer.optimizer.WeightDecay(0.0005))

### Optimize directly
#### Case 1
```python
x = np.random.uniform(-1, 1, (2, 4)).astype('f')
model.cleargrads()
loss = F.sum(model(chainer.Variable(x)))
loss.backward()
optimizer.update()
print(loss)
```

#### Case 2
```
def lossfun(arg1, arg2):
    loss = F.sum(model(arg1 - arg2))
    return loss

x1 = np.random.uniform(-1, 1, (2, 4)).astype('f')
x2 = np.random.uniform(-1, 1, (2, 4)).astype('f')
optimizer.update(lossfun, x1, x2)
```

### Serializer
```
chainer.serializers.save_npz('foo.model', model)
```

```
print(model(x))
chainer.serializers.load_npz('foo.model', model)
model(x)


chainer.serializers.save_npz('foo.state', optimizer)
chainer.serializers.load_npz('foo.state', optimizer)
```

## Get MNIST data

In [80]:
chainer.cuda.get_device_from_id(0).use()

#train, test = chainer.datasets.get_mnist(ndim=3)
train, test = chainer.datasets.get_cifar10()

train_iter = chainer.iterators.SerialIterator(train, batch_size=100, shuffle=True)
test_iter = chainer.iterators.SerialIterator(test, batch_size=100, repeat=False, shuffle=False)

### Example of the classifier
```
class Classifier(chainer.Chain):
    def __init__(self, predictor):
        super(Classifier, self).__init__()
        with self.init_scope():
            slf.predictor = predictor
            
    def __call__(self, x, t):
        y = self.predictor(x)
        loss = F.softmax_cross_entropy(y, t)
        accuracy = F.accuracy(y, t)
        report({'loss': loss, 'accuracy': accuracy}, self)
        return loss
```

In [114]:
LeNet = L.Classifier(LeNetC(10))
AlexNet = L.Classifier(AlexLike())
model_list = [L.GoogLeNet(pretrained_model=None), LeNet, L.GoogLeNet(pretrained_model=None),
              L.VGG16Layers(pretrained_model=None),
              L.ResNet50Layers(pretrained_model=None), L.ResNet152Layers(pretrained_model=None)]

#model_list = [L.VGG16Layers(pretrained_model=None)]
model_list = [AlexNet, LeNet]

epoch_cnt = 300
outdir = './tuto-result/'

gpu_device = 0
for i, model in enumerate(model_list):
    train_iter.reset()
    test_iter.reset()
    
    # Print model name
    predictor = model.predictor if isinstance(model, L.Classifier) else model
    model_name = predictor.__class__.__name__
    print("#### {0}-{1}".format(i, model_name))

    # Optimizer
    model.to_gpu(0); gpu_device=0
    optimizer = chainer.optimizers.SGD()
    optimizer.setup(model)

    # 
    updater = chainer.training.StandardUpdater(train_iter, optimizer, device=gpu_device)
    trainer = chainer.training.Trainer(updater, (epoch_cnt, 'epoch'), out=outdir)

    # Extensions
    trainer.extend(extensions.Evaluator(test_iter, model, device=gpu_device))
    trainer.extend(extensions.LogReport())
    trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'validation/main/loss', 'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))
    trainer.extend(extensions.PlotReport(['main/accuracy', 'validation/main/accuracy'], 'epoch', file_name='acc.png'))
    # trainer.extend(extensions.ParameterStatistics(model))
    # trainer.extend(extensions.ProgressBar())

    # Train
    #start_time = time.time()
    trainer.run()
    #print("  [{0}-{1}] Execution time: {2:.2f} sec".format(i, model_name, time.time() - start_time)), time.sleep(0.01)
    
    chainer.serializers.save_npz(outdir + '{0}-{1}.model'.format(i, model_name), model)


#### 0-AlexLike
epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time


1           2.29069     2.24582               0.11484        0.1441                    12.8079       


2           2.14803     2.04862               0.19156        0.2485                    24.625        


3           1.99016     1.79084               0.25974        0.3562                    36.664        


4           1.835       1.76556               0.32248        0.3601                    48.2481       


...

96          0.148278    0.855417              0.948142       0.7871                    1235.57       


97          0.139731    0.862602              0.951101       0.7929                    1248.95       


98          0.133931    0.894406              0.953081       0.7862                    1261.88       


99          0.133686    0.83356               0.953341       0.7952                    1274.84       


100         0.128363    0.841051              0.955001       0.7955                    1287.61       


In [41]:
x = np.random.rand(32, 1, 28, 28).astype(np.float32)
t = np.random.randint(0, 10, size=(32,)).astype(np.int32)

model = LeNet5(10)
y = model(x)
loss = F.softmax_cross_entropy(y, t)

loss, time.sleep(0.01)

(variable(2.3398814), None)

In [44]:
train[0][0].shape

(784,)

In [45]:
28 * 28 

784